In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from pyvi import ViTokenizer

from sklearn.model_selection import learning_curve
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import re

In [ ]:
!pip install py-stringmatching

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 661 kB 5.1 MB/s 
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055575 sha256=664c0479a4ee7eea5ad22520898f0f940cf2693605f08d04098348332c3623cb
  Stored in directory: /root/.cache/pip/wheels/7e/12/86/46f9378ae730550dd2f89cad50880e70d66cabedd90074f2d2
Successfully built py-stringmatching


In [ ]:
import py_stringmatching as sm

#1 Huấn luyện mô hình Naive Bayes cho các cột có dữ liệu dạng String

In [ ]:
#read File
data = pd.read_csv('/content/drive/MyDrive/My THDL/Bat-dong-san2.csv')
print('Các nhãn và số dữ liệu non-NaN tương ứng')
data.count()

Các nhãn và số dữ liệu non-NaN tương ứng


Số điện thoại     2376
Địa chỉ           2376
Mô tả             2376
Ngày đăng         2376
Số tầng            588
Số phòng ngủ      1267
Tiêu đề           2376
Hướng ban công     536
Pháp lý           1525
Mức giá           2376
Diện tích         2374
Ảnh               2226
Số toilet         1159
Mặt tiền           817
Hướng nhà          820
Mã tin            2376
Ngày hết hạn      2376
Đường vào          810
Nội thất           726
Tên người bán     2376
Original Link     2376
dtype: int64

In [ ]:
pd.set_option('display.max_columns', None)
data.head()

,Số điện thoại,Địa chỉ,Mô tả,Ngày đăng,Số tầng,Số phòng ngủ,Tiêu đề,Hướng ban công,Pháp lý,Mức giá,Diện tích,Ảnh,Số toilet,Mặt tiền,Hướng nhà,Mã tin,Ngày hết hạn,Đường vào,Nội thất,Tên người bán,Original Link
0,0979016016,"Xã Tân Lập, Hàm Thuận Nam, Bình Thuận",\r\n Đất ngay đường dẫn cao tốc...,27/06/2022,NaN,NaN,Bán 10 mẫu ngay mặt tiền đường dẫn cao tốc Ph...,NaN,Sổ đỏ/ Sổ hồng,Thỏa thuận,93252 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,500 m,NaN,33431442,27/06/2022,5 m,NaN,Quách Hương,https://batdongsan.com.vn/ban-dat-xa-tan-lap-1...
1,0979016016,"Đường Quang Trung, Phường Tăng Nhơn Phú B, Quậ...",\r\n Sở hữu đất rộng ngay UBND ...,27/06/2022,NaN,NaN,"Tôi không bán đất, tôi chỉ muốn khoe tôi có 2...",NaN,Sổ đỏ/ Sổ hồng,Thỏa thuận,238 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,7 m,NaN,33886627,27/06/2022,6 m,NaN,Quách Hương,https://batdongsan.com.vn/ban-dat-duong-quang-...
2,0979016016,"Đường ĐT 716, Xã Hoà Thắng, Bắc Bình, Bình Thu...",\r\n Siêu phẩm đất view biển nằ...,27/06/2022,NaN,NaN,Tôi không bán Đất. Tôi chỉ muốn khoe tôi có 3...,NaN,Sổ đỏ/ Sổ hồng,Thỏa thuận,4169 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,45 m,NaN,33154416,27/06/2022,16 m,NaN,Quách Hương,https://batdongsan.com.vn/ban-dat-duong-dt-716...
3,0901478123,"Dự án Richmond Quy Nhơn, Quy Nhơn, Bình Định",\r\n - Tập đoàn Hưng Thịnh mở b...,27/06/2022,NaN,NaN,Hưng Thịnh mở bán 40 lô đất nền đô thị kiểu m...,NaN,Hợp đồng mua bán,5.3 tỷ,100 m²,https://file4.batdongsan.com.vn/resize/200x200...,NaN,NaN,Đông,34232817,27/06/2022,13 m,NaN,Tuấn Trần,https://batdongsan.com.vn/ban-dat-nen-du-an-du...
4,0348666658,"Dự án Mê Linh - Thanh Lâm - Đại Thịnh, Mê Linh...",\r\n Chính chủ hiện nay tôi có ...,27/06/2022,4 tầng,4 phòng,"Quỹ 5 căn góc liền kề, biệt thự trục chính, m...",NaN,NaN,7.3 tỷ,111 m²,https://file4.batdongsan.com.vn/resize/200x200...,5 phòng,NaN,NaN,34232284,27/06/2022,NaN,Nhà xây thô hoàn thiện mặt ngoài.,Đào Tiến Minh,https://batdongsan.com.vn/ban-nha-biet-thu-lie...


In [ ]:
#Chọn các cột có dữ liệu dạng String
data = pd.DataFrame(data[['Địa chỉ', 'Mô tả', 'Tiêu đề', 'Pháp lý', 'Hướng nhà', 'Nội thất', 'Tên người bán']])

In [ ]:
#Mapping các giá trị tên trường thành số
wordsDict = {}
for i in range(len(data.columns)):
  print(data.columns[i] + ': ' + str(i))
  wordsDict[i] = data.columns[i]

Địa chỉ: 0
Mô tả: 1
Tiêu đề: 2
Pháp lý: 3
Hướng nhà: 4
Nội thất: 5
Tên người bán: 6


In [ ]:
X = []
y = []
for i in range(len(data.columns)):
  temp_Arr = data[data.columns[i]].dropna().tolist()
  for j in temp_Arr:
    X.append(j)
    y.append(i)
print(len(X))
print(len(y))

12575
12575


In [ ]:
#Chuyển các xâu về dạng TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
print(X.shape)

(12575, 11372)


In [ ]:
#Chia dữ liệu thành tập train, test
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)
print(len(Y_train))

(10060, 11372)
10060


In [ ]:
#Huấn luyện mô hình
NB = MultinomialNB()
NB.fit(X_train, Y_train)

MultinomialNB()

In [ ]:
print("Acc trên tập train:", NB.score(X_train, Y_train))
print("Acc trên tập test:", NB.score(X_test, Y_test))
print(classification_report(Y_test, NB.predict(X_test)))

Acc trên tập train: 0.8917495029821073
Acc trên tập test: 0.852882703777336
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       461
           1       0.63      0.98      0.77       504
           2       0.93      0.55      0.69       461
           3       1.00      0.95      0.98       305
           4       0.99      1.00      0.99       155
           5       1.00      0.68      0.81       160
           6       1.00      0.82      0.90       469

    accuracy                           0.85      2515
   macro avg       0.92      0.85      0.87      2515
weighted avg       0.89      0.85      0.85      2515



#2. Schema Matching


In [ ]:
def isPhoneFormat(s):
  validate_phone_number_pattern = "^[0-9\-\+]{9,15}$"
  return bool(re.match(validate_phone_number_pattern, s))

In [ ]:
def isTimeFormat(s):
  time_pattern = "^[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}$"
  return bool(re.match(time_pattern, s))

In [ ]:
def URLFormat(s):
  url_pattern = "^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$"
  return bool(re.match(url_pattern, s))

In [ ]:
def isImageFormat(s):
  if ('.jpg' in s) or ('.png' in s):
    return True
  return False

In [ ]:
def isArea(s):
  if (' m²' in s):
    return True
  return False

In [ ]:
def isMeter(s):
  if (' m' in s):
    return True
  return False

In [ ]:
chotot = pd.read_csv('/content/drive/MyDrive/My THDL/Cho-tot2.csv')

In [ ]:
chotot.head()

,Unnamed: 0,Số phòng ngủ:,Chiều ngang:,Mã căn / Mã căn hộ:,Số phòng vệ sinh:,Chi tiết,Dự Án:,Link nguồn,Loại hình nhà ở:,Ảnh nhà,Tình trạng nội thất:,Đặc điểm nhà/đất:,Giấy tờ pháp lý:,Người bán,Diện tích đất:,Liên lạc,Giá/m2:,Chiều dài:,Giá đề xuất,Diện tích sử dụng:,Địa chỉ,Tên bài đăng,Tên phân khu/Lô/Block/Tháp:,Tổng số tầng:,Hướng cửa chính:
0,NaN,2 phòng,4 m,NaN,1 phòng,"Nhà trệt lầu hoàn công, nở hậu, Hẻm 98 - Phạm ...",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-ni...,Nhà ngõ,https://cdn.chotot.com/K1dRMhyumnlHZmFwkweuO6F...,Hoàn thiện cơ bản,NaN,Đã có sổ,Mỹ Kim,38 m²,0943700368,"47,11 triệu/m²",10 m,"1,79 tỷ",NaN,"Đường Phạm Ngũ Lão, Phường An Hòa, Quận Ninh K...",Nhà trệt lầu hẻm 98 Phạm Ngũ Lão,NaN,1,Tây Bắc
1,NaN,3 phòng,NaN,NaN,NaN,Bán nhà Vân Canh 5 TẦNG MỚI Ở NGAY \r\n+Oto tr...,NaN,https://nha.chotot.com/mua-ban-nha-dat-huyen-h...,Nhà ngõ,https://cdn.chotot.com/qgN7JPtEr2_wkCN-NIpfnmE...,NaN,NaN,NaN,Thiện,32 m²,0376369183,75 triệu/m²,NaN,"2,4 tỷ",NaN,"Đường An Trai, Xã Vân Canh, Huyện Hoài Đức, Hà...",BÁN NHÀ VÂN CANH 5 TẦNG Ở NGAY CHỈ 2TY4,NaN,NaN,NaN
2,NaN,2 phòng,4 m,NaN,2 phòng,Chủ cần sang nhượng nhà để di cư sang nước ngo...,NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-bi...,Nhà ngõ,https://cdn.chotot.com/16mWE6EtFcFUjq3SbgeMd1y...,NaN,NaN,Đã có sổ,Mỹ Tiên,46 m²,0938455817,"26,96 triệu/m²",11.5 m,"1,24 tỷ",46 m²,"Đường Lê Văn Quới, Phường Bình Trị Đông A, Quậ...","NHÀ HẺM 1 TRỆT 1 LẦU, LÊ VĂN QUỚI_B.TÂN/46m2,...",NaN,1,NaN
3,NaN,2 phòng,NaN,NaN,2 phòng,"- Vị trí nhà đẹp, lô góc 2 mặt thoáng, căn nhà...",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-do...,Nhà ngõ,https://cdn.chotot.com/rJNmUj6bFC4jD4f2vs3NET-...,Nội thất đầy đủ,NaN,Đã có sổ,HOÀNG XUÂN HOÀ,25 m²,0879571234,104 triệu/m²,NaN,"2,6 tỷ",25 m²,"Ngõ 22 Phố Tôn Thất Tùng, Phường Khương Thượng...",SIÊU PHẨM ĐỐNG ĐA•NHỎ TIỀN•LÔ GÓC•25M2 CHỈ 2....,NaN,3,NaN
4,NaN,2 phòng,5 m,NaN,2 phòng,"Nhà mặt tiền Chiết Ga ( 567 Lê văn khương), p....",NaN,https://nha.chotot.com/mua-ban-nha-dat-quan-12...,Nhà mặt phố,https://cdn.chotot.com/kUbqhbsIInNyHMfb59Bt5rR...,Hoàn thiện cơ bản,NaN,Đã có sổ,Bất Động Sản TOÀN CẦU LAND,161 m²,0985239435,"35,09 triệu/m²",32 m,"5,65 tỷ",161 m²,"Hẻm 567 Đường Lê Văn Khương, Phường Hiệp Thành...","Nhà 5.15MX32M Mặt Tiền Chiết Ga, P.Hiệp Thành...",NaN,1,Nam


In [ ]:
print(chotot.columns)

Index(['Unnamed: 0', 'Số phòng ngủ: ', 'Chiều ngang: ', 'Mã căn / Mã căn hộ: ',
       'Số phòng vệ sinh: ', 'Chi tiết', 'Dự Án: ', 'Link nguồn',
       'Loại hình nhà ở: ', 'Ảnh nhà', 'Tình trạng nội thất: ',
       'Đặc điểm nhà/đất: ', 'Giấy tờ pháp lý: ', 'Người bán',
       'Diện tích đất: ', 'Liên lạc', 'Giá/m2: ', 'Chiều dài: ', 'Giá đề xuất',
       'Diện tích sử dụng: ', 'Địa chỉ', 'Tên bài đăng',
       'Tên phân khu/Lô/Block/Tháp: ', 'Tổng số tầng: ', 'Hướng cửa chính: '],
      dtype='object')


In [ ]:
batdongsan = pd.read_csv('/content/drive/MyDrive/My THDL/Bat-dong-san2.csv')

In [ ]:
def normalize_price(x):   
    if 'tỷ' in x or 'triệu' in x:
      if (x[-2:]) == 'tỷ':
        x = str(float(x[:-3].replace(',', '.'))*1000)
      x = x.replace('tỷ','')
      x = x.replace('triệu','')  
      x = x.replace(' ', '')
      try:
        x = float(x)
        if x > 0:
          return x
        else: 
          return str('Thỏa thuận')
      except:
        return str('Thỏa thuận')
    else:
      return str('Thỏa thuận')

def normalize_square(x):   
  x = x.split(' ')
  if x[0] != 'nan' and len(x[0]) > 0:

    x[0] = x[0].replace(',', '.')
    return float(x[0])

def normalize_room(x):  
  if str(x) != 'Chưa có thông tin': 
    x = x.split(' ')
    if len(x) > 1:
      if x[0] != 'nan':
        return str(int(x[-2]))
    else:
      try:
        x = int(float(x[0]))
        return str(x)
      except:
        return str('Chưa có thông tin')
  return str('Chưa có thông tin')

In [ ]:
chotot['Giá đề xuất'] = chotot['Giá đề xuất'].apply(lambda x: normalize_price(str(x)))
chotot['Diện tích đất: '] = chotot['Diện tích đất: '].apply(lambda x: normalize_square(str(x)))
chotot['Số phòng ngủ: '] = chotot['Số phòng ngủ: '].apply(lambda x: normalize_room(str(x)))      
chotot['Số phòng vệ sinh: '] = chotot['Số phòng vệ sinh: '].apply(lambda x: normalize_room(str(x)))

In [ ]:
batdongsan['Số phỏng ngủ'] = batdongsan['Số phỏng ngủ'].apply(lambda x: normalize_room(str(x)))
batdongsan['Số toilet'] = batdongsan['Số toilet'].apply(lambda x: normalize_room(str(x)))
batdongsan['Diện tích'] = batdongsan['Diện tích'].apply(lambda x: normalize_square(str(x)))
batdongsan['Mức giá'] = batdongsan['Mức giá'].apply(lambda x: normalize_price(str(x)))

In [ ]:
show = pd.DataFrame(chotot[['Số phòng vệ sinh: ', 'Số phòng ngủ: ', 'Diện tích đất: ', 'Giá đề xuất']])
show.head()

,Số phòng vệ sinh:,Số phòng ngủ:,Diện tích đất:,Giá đề xuất
0,1 phòng,2 phòng,38 m²,"1,79 tỷ"
1,NaN,3 phòng,32 m²,"2,4 tỷ"
2,2 phòng,2 phòng,46 m²,"1,24 tỷ"
3,2 phòng,2 phòng,25 m²,"2,6 tỷ"
4,2 phòng,2 phòng,161 m²,"5,65 tỷ"


In [ ]:
show = pd.DataFrame(chotot[['Số phòng vệ sinh: ', 'Số phòng ngủ: ', 'Diện tích đất: ', 'Giá đề xuất']])
show.head()

,Số phòng vệ sinh:,Số phòng ngủ:,Diện tích đất:,Giá đề xuất
0,1,2,38.0,1790.0
1,Chưa có thông tin,3,32.0,2400.0
2,2,2,46.0,1240.0
3,2,2,25.0,2600.0
4,2,2,161.0,5650.0


In [ ]:
alnum_tok = sm.AlphanumericTokenizer()
jac = sm.Jaccard()
def matchingScore(s1, s2, value, p):
  return (1-p)*jac.get_sim_score(alnum_tok.tokenize(s1), alnum_tok.tokenize(s2)) + p*value

In [ ]:
chotot_columns = chotot.columns
mapping = {}
for i in range(len(chotot_columns)):
    temp_Arr = chotot[chotot.columns[i]].dropna().tolist()
    for j in temp_Arr:
      if (isPhoneFormat(str(j))):
        mapping[chotot_columns[i]] = ['Số điện thoại', matchingScore(chotot_columns[i], 'Số điện thoại', 1, 0.7)]
        break
      if (isImageFormat(str(j))):
        mapping[chotot_columns[i]] = ['Ảnh', matchingScore(chotot_columns[i], 'Ảnh', 1, 0.7)]
        break
      if (URLFormat(str(j))):
        mapping[chotot_columns[i]] = ['Original Link', matchingScore(chotot_columns[i], 'Original Link', 1, 0.7)]
        break

In [ ]:
mapping['Link nguồn'] = ['Original Link', 0.7749999999999999]
mapping['Ảnh nhà'] = ['Ảnh', 1.0]
mapping['Liên lạc'] = ['Số điện thoại', 0.7428571428571428]
mapping['Diện tích đất: '] = ['Diện tích', 1.0]
mapping['Giá đề xuất'] = ['Giá', 0.8666666666666666]
mapping['Số phòng ngủ: '] = ['Số phòng ngủ', 1.0]
mapping['Số phòng vệ sinh: '] = ['Số phòng ngủ', 0.88]
mapping['Người bán'] = ['Tên người bán', 0.6719810209460296]
mapping['Địa chỉ'] = ['Địa chỉ', 0.8299552557376733]
mapping['Giấy tờ pháp lý: '] = ['Pháp lý', 0.6313472153887432]
mapping['Tên bài đăng'] = ['Tiêu đề', 0.5439927952479585]
mapping['Chi tiết'] = ['Mô tả', 0.6132754312790492]
mapping['Tình trạng nội thất: '] = ['Nội thất', 0.6257582521429083]
mapping['Hướng cửa chính'] = ['Hướng nhà', 0.6774031765866391]

In [ ]:
for item in mapping:
  print(str(item) + '----' + str(mapping[item]))

Link nguồn----['Original Link', 0.7749999999999999]
Ảnh nhà----['Ảnh', 1.0]
Liên lạc----['Số điện thoại', 0.7428571428571428]
Diện tích đất: ----['Diện tích', 1.0]
Giá đề xuất----['Giá', 0.8666666666666666]
Số phòng ngủ: ----['Số phòng ngủ', 1.0]
Số phòng vệ sinh: ----['Số phòng ngủ', 0.88]
Người bán----['Tên người bán', 0.6719810209460296]
Địa chỉ----['Địa chỉ', 0.8299552557376733]
Giấy tờ pháp lý: ----['Pháp lý', 0.6313472153887432]
Tên bài đăng----['Tiêu đề', 0.5439927952479585]
Chi tiết----['Mô tả', 0.6132754312790492]
Tình trạng nội thất: ----['Nội thất', 0.6257582521429083]
Hướng cửa chính----['Hướng nhà', 0.6774031765866391]
